# Benchmarking Different Quantum Hardware Devices using Grover's Algorithm

## Code Steps:
1. [Libraries, Parameters and Global Variables](#variables)
2. [Constructing Grover's Circuit using Qiskit](#grover_circuit)
3. [Running Grover's Circuits on IBM Hardware](#ibm)
4. [Preparing Grover's Circuits on AWS Braket](#aws_circuits)
    1. [Running Grover's Circuit on IonQ](#ionq)
    2. [Running Grover's Circuit on Rigetti](#rigetti)
5. [Collecting Results from Queued Jobs/Tasks](#results)
5. [Saving Data to CSV files](#save_data)

## 1. Libraries, Parameters and Global Variables <a name="variables"></a>

In [1]:
# Built-in modules
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import Braket libraries
from braket.aws import AwsDevice, AwsQuantumTask
from braket.circuits import Circuit as awsCircuit, circuit
from braket.devices import Devices, LocalSimulator

# Imports from Qiskits
from qiskit import QuantumCircuit
from qiskit.visualization import plot_distribution

# Imports from Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService

# Import helper functions
from helper import generate_short_key


In [2]:
## Initialize global variables to store input parameters and output

# Initialize the DataFrame for input parameters
param_df = pd.DataFrame(columns=['key', 'target_states', 'quantum_circuit', 'num_qubits', 'depth'])

# Initialize the DataFrame for storing output
output_df = pd.DataFrame(columns=['key', 'device_name', 'transpiled_depth', 'job_id', 'counts', 'execution_time'])

print(param_df)
print(output_df)

# Sample key generation
key = generate_short_key()
print(f"\nGenerated Key: {key}")

# Initialize lists to store IDs of jobs/tasks running on hardware
tasks_ibm = []
tasks_ionq = []
tasks_rigetti = []

Empty DataFrame
Columns: [key, target_states, quantum_circuit, num_qubits, depth]
Index: []
Empty DataFrame
Columns: [key, device_name, transpiled_depth, job_id, counts, execution_time]
Index: []

Generated Key: 58a0a33b


## 2. Constructing Grover's Circuit using Qiskit <a name="grover_circuit"></a>
1. This section takes an upper limit to the number of qubits $(n_{max})$ in a circuit, describing the size of the database. 
2. A random string of n-bit `target_states`, i.e., states to be searched in the database, is generated to construct the Grover's Oracle. To reduce circuit depth, we set the number of possible target_states to be atmost 4.
3. A total of $n_{max}$ Grover's circuits are generated, each with increasing number of qubits from 2 to $n_{max}$.
4. This list of target states and Grover's circuits are stored in the `param_df` DataFrame.

In [3]:
from helper import generate_random_n_bit_strings

# Maximum number of qubits
n_max = 2

# Generate random target_states for the Oracle
target_states = []

for n in range(2, n_max+1):
    states = generate_random_n_bit_strings(n)
    target_states.append(states)
print(f"Target states: \n {target_states}")


Target states: 
 [['10']]


In [4]:
from grover import grover_circuit

# Generate n_max Grover's circuits and storing in Data Frame
idx = 0
param_df.drop(param_df.index, inplace=True)

for states in target_states:
    circuit = grover_circuit(states)
    key = generate_short_key()
    param_df.loc[idx] = [key, list(states), circuit, circuit.num_qubits, circuit.depth()]
    idx = idx + 1

print(param_df)

# Print the Grover's circuits
for qc in param_df["quantum_circuit"]:
    print(qc)

        key target_states                                    quantum_circuit  \
0  1aa22e93          [10]  ((Instruction(name='h', num_qubits=1, num_clbi...   

   num_qubits  depth  
0           2      3  
        ┌───┐┌────┐ ░ ┌─┐   
   q_0: ┤ H ├┤0   ├─░─┤M├───
        ├───┤│  Q │ ░ └╥┘┌─┐
   q_1: ┤ H ├┤1   ├─░──╫─┤M├
        └───┘└────┘ ░  ║ └╥┘
meas: 2/═══════════════╩══╩═
                       0  1 


## 3. Running Grover's Circuits on IBM Hardware <a name="ibm"></a>

In [5]:
# Save an IBM Quantum account as the default account, and load saved credentials
from ibm_qiskit.config import api_key 

QiskitRuntimeService.save_account(
    channel="ibm_quantum", token=api_key, set_as_default=True, overwrite=True
)
service = QiskitRuntimeService()
# service.backends()  # List all available backends
backend = service.backend("ibm_fez")    # Choosing 'ibm_fez' backend 
print(f"Backend Selected: {backend}")

Backend Selected: <IBMBackend('ibm_fez')>


In [6]:
from ibm_qiskit.run import aer_without_noise, aer_noisy, run_backend_job

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))

# # Run circuits on Aer simulator (without noise)
output = aer_without_noise(key_circuit)
for key, depth, counts, time in output:
    output_df.loc[len(output_df)] = [key, 'aer_sim', depth, '', counts, time]
    

# Run circuits on noisy Aer simulator
output = aer_noisy(key_circuit, backend)
for key, depth, counts, time in output:
    output_df.loc[len(output_df)] = [key, 'aer_noisy', depth, '', counts, time]


# Run circuits on actual IBM backend
ibm_device = backend.name
output = run_backend_job(key_circuit, backend)
for key, job_id, depth in output:
    output_df.loc[len(output_df)] = [key, ibm_device, depth, job_id, '', '']

In [10]:
# Print all entries for IBM Hardware
ibm_df = output_df[output_df['device_name'].isin(['aer_sim', 'aer_noisy', ibm_device])]
ibm_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
0,1aa22e93,aer_sim,6,,{'10': 1024},2.02355
1,1aa22e93,aer_noisy,12,,"{'01': 6, '10': 970, '11': 12, '00': 36}",14.710615
2,1aa22e93,ibm_fez,9,cxapcgjrkac00089q27g,,


## 4. Preparing Grover's Circuits on AWS Braket <a name="aws_circuits"></a>

In [17]:
# Use Braket SDK Cost Tracking to estimate the cost to run this task
from braket.tracking import Tracker

t = Tracker().start()

In [ ]:
# Step 2: Convert Qiskit Circuit to Braket Circuit
braket_circuit = awsCircuit().from_qiskit(qiskit_circuit)

### A. Running Grover's circuits on IonQ <a name="ionq"></a>

### B. Running Grover's circuits on Rigetti <a name="rigetti"></a>

## 5. Collecting Results from Queued Jobs/Tasks <a name="results"></a>


In [11]:
# Print results from IBM simulators
aer_df = output_df[output_df['device_name'].isin(['aer_sim', 'aer_noisy'])]
aer_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
0,1aa22e93,aer_sim,6,,{'10': 1024},2.02355
1,1aa22e93,aer_noisy,12,,"{'01': 6, '10': 970, '11': 12, '00': 36}",14.710615


In [ ]:
# Get results from real IBM backend
from helper import update_dataframe_with_job_results

update_dataframe_with_job_results(service, output_df, ibm_device)

# Print results from IBM device
ibm_df = output_df[output_df['device_name'] == ibm_device]
ibm_df

Job cxapcgjrkac00089q27g is not completed yet. Current status: QUEUED


,key,device_name,transpiled_depth,job_id,counts,execution_time
2,1aa22e93,ibm_fez,9,cxapcgjrkac00089q27g,,


In [ ]:
# Print results from AWS Simulators


In [ ]:
# Get results from AWS linked hardware

# Print results from AWS linked hardware

## 6. Saving Data to CSV files<a name="save_data"></a>

In [14]:
# Save DataFrame to CSV files
import os
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

subfolder = 'saved_data'

# Create the subfolder if it does not exist
os.makedirs(subfolder, exist_ok=True)

# Save DataFrames to CSV files
param_name = os.path.join(subfolder, f"parameters_{timestamp}.csv")
param_df.to_csv(param_name, index=False)  # index=False prevents extra index column

output_name = os.path.join(subfolder, f"output_{timestamp}.csv")
output_df.to_csv(output_name, index=False)

In [ ]:
# Read from CSV files into DataFrames

param_df = pd.read_csv('parameters.csv')
output_df = pd.read_csv('output.csv')
print(param_df)
print(output_df)